Pranavi Vasa - 1806152, Siddharth Shyamsunder - 1802772

# Task - 2

importing required libraries

In [495]:
import pandas as pd

In [496]:
import numpy as np
import re

In [497]:
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

In [498]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()

In [499]:
from sklearn.pipeline import Pipeline

In [500]:
from sklearn.feature_extraction.text import TfidfTransformer

In [501]:
import nltk

In [502]:
from sklearn.model_selection import train_test_split

In [503]:
import string

In [504]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report 

In [505]:
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding
from keras.layers import LSTM
from nltk.tokenize import word_tokenize 
from gensim import corpora

Reading train data

In [506]:
df_train = pd.read_csv('train.tsv', sep='\t')

Checking how many different kind of sentiments present in train data

In [507]:
df_train.Sentiment.value_counts()

2    79582
3    32927
1    27273
4     9206
0     7072
Name: Sentiment, dtype: int64

In [508]:
raw_docs_train = df_train['Phrase'].values

In [509]:
raw_docs_train

array(['A series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story .',
       'A series of escapades demonstrating the adage that what is good for the goose',
       'A series', ..., 'avuncular chortles', 'avuncular', 'chortles'],
      dtype=object)

In [510]:
sentiment_train = df_train['Sentiment'].values

In [511]:
sentiment_train

array([1, 2, 2, ..., 3, 2, 2], dtype=int64)

Finding different number of labels present

In [512]:
num_labels = len(np.unique(sentiment_train))

In [513]:
num_labels

5

In [514]:
stop_words = set(stopwords.words('english'))

In [515]:
#Converting to lower case
df_train['Phrase'] = df_train['Phrase'].str.lower()

In [516]:
df_train['Phrase'] = df_train['Phrase'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

Removing stop words

In [517]:
df_train['tokezines_sents'] = df_train['Phrase'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

In [518]:
df_train['tokezines_sents'] = df_train.apply(lambda x: nltk.word_tokenize(x['tokezines_sents']),axis=1)

In [519]:
stemmer = SnowballStemmer('english')


Applying stemmer

In [520]:
df_train['tokezines_sents'] = df_train['tokezines_sents'].apply(lambda x: [stemmer.stem(y) for y in x])

In [521]:
df_train['tokezines_sents'] = df_train['tokezines_sents'].apply(lambda x: ' '.join(x))

In [522]:
y = df_train['Sentiment']


Loading test data and applying preprocessing steps as applied on train data

In [523]:
df_test= pd.read_csv('test.tsv', sep='\t')
df_test['Phrase'] = df_test['Phrase'].str.lower()
df_test['Phrase'] = df_test['Phrase'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
df_test['tokezines_sents'] = df_test['Phrase'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
df_test['tokezines_sents'] = df_test.apply(lambda x: nltk.word_tokenize(x['tokezines_sents']),axis=1)
df_test['tokezines_sents'] = df_test['tokezines_sents'].apply(lambda x: [stemmer.stem(y) for y in x])
df_test['tokezines_sents'] = df_test['tokezines_sents'].apply(lambda x: ' '.join(x))

df_test.head()

,PhraseId,SentenceId,Phrase,tokezines_sents
0,156061,8545,an intermittently pleasing but mostly routine ...,intermitt pleas most routin effort
1,156062,8545,an intermittently pleasing but mostly routine ...,intermitt pleas most routin effort
2,156063,8545,an,
3,156064,8545,intermittently pleasing but mostly routine effort,intermitt pleas most routin effort
4,156065,8545,intermittently pleasing but mostly routine,intermitt pleas most routin


Combining train and test data for training tfidf vectoriser.

In [524]:
total_text = [df_train,df_test]

In [525]:
result=pd.concat(total_text,sort=True)

In [526]:
result.head()

,Phrase,PhraseId,SentenceId,Sentiment,tokezines_sents
0,a series of escapades demonstrating the adage ...,1,1,1.0,seri escapad demonstr adag good goos also good...
1,a series of escapades demonstrating the adage ...,2,1,2.0,seri escapad demonstr adag good goos
2,a series,3,1,2.0,seri
3,a,4,1,2.0,
4,series,5,1,2.0,seri


In [527]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

In [528]:
#Fitting vectoriser
vectorizer.fit(result['tokezines_sents'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [529]:
X = vectorizer.transform(df_train['tokezines_sents'])

In [530]:
#Splitting the data to test the accuracy of the model before submitting it to the kaggle
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=0)

Training and testing MultinomialNB classifier

In [531]:
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

model = MultinomialNB()

In [532]:
model.fit(x_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [533]:
predicted = model.predict(x_test)

In [534]:
print(metrics.classification_report(y_test, predicted))

              precision    recall  f1-score   support

           0       0.61      0.04      0.07      1504
           1       0.52      0.25      0.34      5453
           2       0.60      0.91      0.72     15982
           3       0.53      0.36      0.43      6441
           4       0.64      0.05      0.09      1832

   micro avg       0.59      0.59      0.59     31212
   macro avg       0.58      0.32      0.33     31212
weighted avg       0.58      0.59      0.53     31212



In [535]:
print("accuracy score:",accuracy_score(predicted,y_test))

accuracy score: 0.5860886838395489


In [536]:
df_train.head()

,PhraseId,SentenceId,Phrase,Sentiment,tokezines_sents
0,1,1,a series of escapades demonstrating the adage ...,1,seri escapad demonstr adag good goos also good...
1,2,1,a series of escapades demonstrating the adage ...,2,seri escapad demonstr adag good goos
2,3,1,a series,2,seri
3,4,1,a,2,
4,5,1,series,2,seri


In [537]:
classes = np.unique(y)
classes = classes.tolist()

Training and testing Perceptron classifier

In [538]:
from sklearn.linear_model import Perceptron
per = Perceptron(verbose=10, n_jobs=-1, max_iter=5)
per.partial_fit(x_train, y_train, classes)
print(classification_report(y_pred=per.predict(x_test), y_true=y_test, labels=classes))
print(accuracy_score(y_pred=per.predict(x_test), y_true=y_test))

-- Epoch 1-- Epoch 1-- Epoch 1
-- Epoch 1

-- Epoch 1

Norm: 62.34, NNZs: 9191, Bias: -0.810000, T: 124848, Avg. loss: 0.098411
Total training time: 0.03 seconds.
Norm: 75.26, NNZs: 10783, Bias: 0.670000, T: 124848, Avg. loss: 0.157216Norm: 60.09, NNZs: 9342, Bias: -0.650000, T: 124848, Avg. loss: 0.117218
Total training time: 0.03 seconds.

Total training time: 0.03 seconds.
Norm: 42.42, NNZs: 5379, Bias: -1.340000, T: 124848, Avg. loss: 0.033222Norm: 41.01, NNZs: 5450, Bias: -1.140000, T: 124848, Avg. loss: 0.026477
Total training time: 0.05 seconds.

Total training time: 0.06 seconds.
              precision    recall  f1-score   support

           0       0.28      0.33      0.30      1504
           1       0.41      0.37      0.39      5453
           2       0.69      0.68      0.68     15982
           3       0.41      0.43      0.42      6441
           4       0.31      0.34      0.33      1832

   micro avg       0.54      0.54      0.54     31212
   macro avg       0.42  

C:\Users\Chinnari\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in Perceptron in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   3 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished


Training and testing SGDClassifier

In [539]:
from sklearn.linear_model import SGDClassifier
sgd = SGDClassifier()
sgd.partial_fit(x_train, y_train, classes);
print(classification_report(y_pred=sgd.predict(x_test), y_true=y_test, labels=classes));
print(accuracy_score(y_pred=sgd.predict(x_test), y_true=y_test))

              precision    recall  f1-score   support

           0       0.51      0.09      0.15      1504
           1       0.47      0.12      0.19      5453
           2       0.58      0.95      0.72     15982
           3       0.51      0.24      0.33      6441
           4       0.49      0.14      0.22      1832

   micro avg       0.57      0.57      0.57     31212
   macro avg       0.51      0.31      0.32     31212
weighted avg       0.54      0.57      0.49     31212

0.5674740484429066


In [540]:
 df_test_vec = vectorizer.transform(df_test['tokezines_sents'])

In [541]:
predicted = model.predict(df_test_vec)

In [543]:
final = pd.DataFrame(columns=['PhraseId','Sentiment'])
final['PhraseId'] = df_test['PhraseId']
final['Sentiment'] = predicted
final

,PhraseId,Sentiment
0,156061,2
1,156062,2
2,156063,2
3,156064,2
4,156065,2
5,156066,3
6,156067,3
7,156068,3
8,156069,3
9,156070,2


In [544]:
final.to_csv('ce807_assignment2_pranavi_vasa_siddharth_.csv',index=False)

In [545]:
train_data = df_train.apply(lambda x: nltk.word_tokenize(x['tokezines_sents']),axis=1)

In [546]:
train_data_dt = []
for x in train_data:
    train_data_dt.append(x)
#train_data_dt

In [547]:
test_data = df_test.apply(lambda x: nltk.word_tokenize(x['tokezines_sents']),axis=1)

In [548]:
test_data_dt = []
for x in test_data:
    test_data_dt.append(x)

In [549]:
processed_docs_all = np.concatenate((train_data_dt, test_data_dt), axis=0)

dictionary = corpora.Dictionary(processed_docs_all)
dictionary_size = len(dictionary.keys())

In [550]:
def convert_to_token(doc_list):
    word_ids = []
    id_length = []
    
    for x in doc_list:
        ids = [dictionary.token2id[word] for word in x]
        word_ids.append(ids)
        id_length.append(len(ids))
    return word_ids, id_length
word_id_train, word_id_len_train = convert_to_token(train_data_dt)
word_id_test, word_id_len_test = convert_to_token(test_data_dt)
word_id_len = word_id_len_train + word_id_len_test


 
seq_len = np.round((np.mean(word_id_len) + 2*np.std(word_id_len))).astype(int)

#pad sequences
word_id_train = sequence.pad_sequences(np.array(word_id_train), maxlen=seq_len)
word_id_test = sequence.pad_sequences(np.array(word_id_test), maxlen=seq_len)
y_train_enc = np_utils.to_categorical(sentiment_train, num_labels)

In [552]:
word_ids

[1300, 3647]

In [553]:
word_id_train

array([[    7,     8,     1, ...,     2,     9,    13],
       [    0,     0,     0, ...,     0,     7,     8],
       [    0,     0,     0, ...,     0,     0,    12],
       ...,
       [    0,     0,     0, ...,     0, 11633, 11634],
       [    0,     0,     0, ...,     0,     0, 11633],
       [    0,     0,     0, ...,     0,     0, 11634]])

In [554]:
model = Sequential()
model.add(Embedding(dictionary_size, 128, dropout=0.2))
model.add(LSTM(128, dropout_W=0.2, dropout_U=0.2))
    
model.add(Dense(num_labels))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

C:\Users\Chinnari\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  
C:\Users\Chinnari\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(128, dropout=0.2, recurrent_dropout=0.2)`
  This is separate from the ipykernel package so we can avoid doing imports until


In [559]:
x_train, x_test, y_train, y_test = train_test_split(word_id_train, y_train_enc, test_size = 0.2, random_state=0)

In [566]:
model.fit(x_train, y_train, nb_epoch=1, batch_size=256, verbose=1)

Epoch 1/1
124848/124848 [==============================] - 70s - loss: 0.7314 - acc: 0.6933    


In [567]:
y_pred = model.predict_classes(x_test)

31136/31212 [============================>.] - ETA: 0s

In [569]:
y_pred

array([2, 4, 2, ..., 1, 1, 3], dtype=int64)

In [571]:
y_pred =  np_utils.to_categorical(y_pred, num_labels)

In [572]:
y_pred

array([[0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0.],
       ...,
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0.]])

In [570]:
y_test

array([[0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0.],
       ...,
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0.]])

In [574]:
print(classification_report(y_pred, y_test));
print(accuracy_score(y_pred, y_test))

              precision    recall  f1-score   support

           0       0.30      0.57      0.40       796
           1       0.59      0.54      0.57      5974
           2       0.79      0.76      0.78     16592
           3       0.57      0.56      0.56      6480
           4       0.42      0.56      0.48      1370

   micro avg       0.66      0.66      0.66     31212
   macro avg       0.53      0.60      0.56     31212
weighted avg       0.68      0.66      0.67     31212
 samples avg       0.66      0.66      0.66     31212

0.6641996667948225


In [575]:
test_pred = model.predict_classes(word_id_test)

66292/66292 [==============================] - 11s    


In [576]:
final = pd.DataFrame(columns=['PhraseId','Sentiment'])
final['PhraseId'] = df_test['PhraseId']
final['Sentiment'] = test_pred
final

,PhraseId,Sentiment
0,156061,3
1,156062,3
2,156063,2
3,156064,3
4,156065,3
5,156066,3
6,156067,3
7,156068,2
8,156069,2
9,156070,2


In [577]:
final.to_csv('ce807_assignment2_pranavi_vasa_siddharth_.csv',index=False)